In [1]:
import warnings
warnings.filterwarnings("ignore", "is_categorical_dtype")
warnings.filterwarnings("ignore", "use_inf_as_na")

from analyzer import Analyzer
from tools import *
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

In [5]:
vi_data = {
    'dataset': [],
    'num': [],
    'vi': [],
}

for dataset in ['dogs3']:
    num_vs_time_per_question = {}
    demo_time = []
    nums = []
    num_vs_error = []
    for num in [2, 3, 4, 5, 6, 7, 8]:
        file = f'sd-rq-wp-{num}-{dataset}-2023'
        np.random.seed(0)
        analyer = Analyzer("./data/{}.json".format(file),
                           size_of_dataset(dataset), retrieve_true_adj_matrix(dataset)(), radio=False, dataset=dataset, practice=True)
        analyer.raw_data['demo'].demo_completion_time
        time_per_question = analyer.times
        demo_completion_times = analyer.demo_times

        workers_edge_error_rate_first_half = []
        workers_edge_error_rate_second_half = []

        for answers in analyer.workers_answers:
            true_adj_mat = answers.true_adj_matrix

            num_correct_first_half = 0
            num_incorrect_first_half = 0
            num_correct_second_half = 0
            num_incorrect_second_half = 0
            halves = len(answers.worker_answers) // 2
            edge_errors_first_half = []
            edge_errors_second_half = []

            for answer in answers.worker_answers[:halves]:
                for cluster in answer.clusters:
                    for i, j in combinations(sorted(cluster), 2):
                        if true_adj_mat[i, j] == 1:
                            num_correct_first_half += 1
                            edge_errors_first_half.append(0)
                        else:
                            num_incorrect_first_half += 1
                            edge_errors_first_half.append(1)
                for cluster_x, cluster_y in combinations(answer.clusters, 2):
                    for i, j in product(cluster_x, cluster_y):
                        if true_adj_mat[i, j] == 0:
                            num_correct_first_half += 1
                            edge_errors_first_half.append(0)
                        else:
                            num_incorrect_first_half += 1
                            edge_errors_first_half.append(1)

            for answer in answers.worker_answers[halves:]:
                for cluster in answer.clusters:
                    for i, j in combinations(sorted(cluster), 2):
                        if true_adj_mat[i, j] == 1:
                            num_correct_second_half += 1
                            edge_errors_second_half.append(0)
                        else:
                            num_incorrect_second_half += 1
                            edge_errors_second_half.append(1)
                for cluster_x, cluster_y in combinations(answer.clusters, 2):
                    for i, j in product(cluster_x, cluster_y):
                        if true_adj_mat[i, j] == 0:
                            num_correct_second_half += 1
                            edge_errors_second_half.append(0)
                        else:
                            num_incorrect_second_half += 1
                            edge_errors_second_half.append(1)
            
            edge_error_rate_fist_half = num_incorrect_first_half / (num_correct_first_half + num_incorrect_first_half)
            edge_error_rate_second_half = num_incorrect_second_half / (num_correct_second_half + num_incorrect_second_half)
            workers_edge_error_rate_first_half.append(edge_error_rate_fist_half)
            workers_edge_error_rate_second_half.append(edge_error_rate_second_half)
        
        workers_edge_error_rate_first_half = np.array(workers_edge_error_rate_first_half)
        workers_edge_error_rate_second_half = np.array(workers_edge_error_rate_second_half)

        # print("first_half:", np.mean(workers_edge_error_rate_first_half), np.std(workers_edge_error_rate_first_half))
        # print("second_half:", np.mean(workers_edge_error_rate_second_half), np.std(workers_edge_error_rate_second_half)) 

        diff = workers_edge_error_rate_first_half - workers_edge_error_rate_second_half
        print(f"{diff.mean():.3f} +/- {diff.std():.3f}")

        print()

0.017 +/- 0.160

-0.011 +/- 0.122

0.009 +/- 0.103

0.009 +/- 0.089

-0.002 +/- 0.098

-0.001 +/- 0.099

0.005 +/- 0.099

